In [1]:
import pandas as pd
import os
import yfinance as yf

In [56]:

def analyze_spike_or_drop(data):
    # Calculate daily returns
    data = data.copy() 
    
    data['Daily Return'] = data['Close'].pct_change()

    # Find dates where the stock spiked up by 50% or more or dropped by 50% or more
    significant_dates = data[data['Daily Return'].abs() >= 0.1].index

    days_after_list = [1, 2, 3, 4, 5, 6, 7, 14, 28]
    results_df_list = []

    for significant_date in significant_dates:
        results = {'day0': data.loc[significant_date, 'Daily Return']}
        for days_after in days_after_list:
            try:
                price_before = data.loc[significant_date, 'Adj Close']
                price_after = data.loc[data.index[data.index.get_loc(significant_date) + days_after], 'Adj Close']
                price_change = (price_after - price_before) / price_before
                results[f"{days_after} days after"] = price_change
            except:
                results[f"{days_after} days after"] = None

        results_df = pd.DataFrame(results, index=[significant_date])
        results_df_list.append(results_df)

    if not results_df_list:
        print("No significant movements detected.")
        return None

    final_df = pd.concat(results_df_list)

    def color_cells(val):
        if val is None:
            return 'background-color: #D3D3D3'  # Gray for None values
        elif val >= 0:
            return 'background-color: green'
        else:
            return 'background-color: red'

    styled_df = final_df.style.applymap(color_cells).format("{:.2%}")

    # Calculate the average percentage change for each time frame
    avg_changes = final_df.mean()

    # Calculate the frequency of positive returns for each time frame
    positive_freq = (final_df > 0).sum() / final_df.count()

    # Calculate the frequency of negative returns for each time frame
    negative_freq = (final_df < 0).sum() / final_df.count()

    summary_df = pd.DataFrame({
        'Average Change': avg_changes,
        'Frequency of Positive Returns': positive_freq,
        'Frequency of Negative Returns': negative_freq
    })

    # Formatting the summary DataFrame
    summary_styled = summary_df.style.format({
        'Average Change': "{:.2%}",
        'Frequency of Positive Returns': "{:.2%}",
        'Frequency of Negative Returns': "{:.2%}"
    })
    
    # Build the results list
    results_list = []
    for index, row in final_df.iterrows():
        # Determine the direction based on the 'Daily Return' of 'day0'
        direction = 'up' if data.loc[index, 'Daily Return'] >= 0 else 'down'
        
        for col in final_df.columns:
            days_after = 0 if col == 'day0' else int(col.split()[0])
            return_value = row[col]
            results_list.append({
                'Direction': direction,
                'Days After': days_after,
                'Return': return_value
            })
            
    return styled_df, summary_styled, results_list



In [82]:
tickers = ['AAPL', 'MCD', 'TSLA', 'NVDA']


all_results = []

for ticker in tickers:
    try:
        # Fetch data from yfinance for the past 10 years (modify the period if needed)
        data = yf.download(ticker, period="10y")[['Close', 'Adj Close']]
        print(f"Analyzing {ticker}...")
        result = analyze_spike_or_drop(data)
        if result:
            display_df, summary_styled, results_for_ticker = result
            all_results.extend(results_for_ticker)
            display(display_df)

    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")




[*********************100%%**********************]  1 of 1 completed
Analyzing AAPL...


,day0,1 days after,2 days after,3 days after,4 days after,5 days after,6 days after,7 days after,14 days after,28 days after
2020-03-13 00:00:00,11.98%,-12.86%,-9.03%,-11.26%,-11.94%,-17.53%,-19.28%,-11.18%,-11.89%,-1.06%
2020-03-16 00:00:00,-12.86%,4.40%,1.84%,1.06%,-5.35%,-7.37%,1.93%,1.37%,-0.33%,16.83%
2020-03-24 00:00:00,10.03%,-0.55%,4.68%,0.35%,3.21%,3.00%,-2.42%,-0.79%,16.27%,18.75%
2020-07-31 00:00:00,10.47%,2.52%,3.20%,3.58%,7.19%,4.76%,6.28%,3.12%,11.51%,7.00%


[*********************100%%**********************]  1 of 1 completed
Analyzing MCD...


,day0,1 days after,2 days after,3 days after,4 days after,5 days after,6 days after,7 days after,14 days after,28 days after
2020-03-16 00:00:00,-15.88%,-0.93%,-7.86%,0.33%,-0.35%,-7.99%,8.68%,9.38%,7.60%,23.50%
2020-03-24 00:00:00,18.13%,0.64%,3.33%,1.27%,3.82%,2.10%,-2.33%,-0.28%,13.61%,12.30%
2020-04-06 00:00:00,10.42%,-0.82%,0.25%,3.76%,1.74%,3.93%,0.45%,1.39%,5.00%,-1.82%


[*********************100%%**********************]  1 of 1 completed
Analyzing TSLA...


,day0,1 days after,2 days after,3 days after,4 days after,5 days after,6 days after,7 days after,14 days after,28 days after
2013-11-06 00:00:00,-14.51%,-7.54%,-8.74%,-4.27%,-8.84%,-8.24%,-8.97%,-10.39%,-20.28%,0.86%
2013-11-18 00:00:00,-10.24%,3.71%,-0.39%,0.43%,-0.16%,-0.61%,-0.89%,4.41%,16.47%,25.38%
2013-12-03 00:00:00,16.53%,-3.97%,-2.92%,-5.07%,-2.14%,-1.73%,-3.49%,1.91%,-0.79%,11.45%
2014-01-14 00:00:00,15.74%,1.77%,6.01%,5.42%,9.56%,10.72%,12.54%,8.27%,10.83%,53.78%
2014-02-25 00:00:00,13.94%,2.02%,1.83%,-1.29%,1.03%,2.76%,1.88%,1.99%,-5.65%,-14.42%
2014-05-08 00:00:00,-11.30%,2.05%,3.40%,6.48%,6.74%,5.60%,7.26%,9.80%,17.72%,27.17%
2015-11-04 00:00:00,11.17%,0.06%,0.32%,-2.72%,-6.53%,-5.42%,-8.07%,-10.55%,-5.78%,-4.55%
2016-06-22 00:00:00,-10.45%,-0.13%,-1.78%,0.96%,2.61%,6.88%,7.94%,10.09%,13.15%,15.53%
2018-08-02 00:00:00,16.19%,-0.39%,-2.16%,8.59%,5.95%,0.83%,1.70%,1.97%,-7.98%,-16.88%
2018-08-07 00:00:00,10.99%,-2.43%,-7.14%,-6.34%,-6.10%,-8.41%,-10.77%,-11.62%,-15.89%,-22.32%


[*********************100%%**********************]  1 of 1 completed
Analyzing NVDA...


,day0,1 days after,2 days after,3 days after,4 days after,5 days after,6 days after,7 days after,14 days after,28 days after
2015-08-07 00:00:00,12.37%,3.48%,2.96%,3.22%,2.35%,2.44%,1.78%,0.86%,-1.11%,1.86%
2015-11-06 00:00:00,13.86%,-0.44%,-2.35%,-3.30%,-3.65%,-5.55%,-3.65%,-3.65%,-0.13%,3.94%
2016-05-13 00:00:00,15.21%,2.95%,3.17%,5.81%,6.27%,8.17%,8.35%,10.98%,13.72%,18.63%
2016-11-11 00:00:00,29.81%,-4.92%,-2.02%,4.16%,5.02%,6.13%,5.70%,6.46%,0.70%,21.94%
2017-05-10 00:00:00,17.83%,4.30%,5.44%,10.73%,12.80%,5.30%,9.71%,12.25%,19.14%,29.65%
2018-11-16 00:00:00,-18.76%,-12.00%,-9.34%,-11.99%,-11.82%,-6.92%,-6.51%,-2.65%,-7.55%,-18.73%
2018-11-19 00:00:00,-12.00%,3.03%,0.01%,0.21%,5.77%,6.24%,10.62%,8.86%,2.51%,-5.77%
2019-01-28 00:00:00,-13.82%,-4.64%,-0.45%,4.16%,4.87%,8.09%,8.65%,10.86%,14.01%,9.26%
2020-03-12 00:00:00,-12.24%,11.34%,-9.20%,0.44%,-6.24%,-1.54%,-4.88%,-1.67%,12.37%,32.29%
2020-03-13 00:00:00,11.34%,-18.45%,-9.79%,-15.79%,-11.57%,-14.57%,-11.69%,3.46%,6.07%,17.92%


In [83]:
df = pd.DataFrame(all_results, columns=['Direction', 'Days After', 'Return'])

summary = {
    'average stocks up': df[df['Direction'] == 'up'].groupby('Days After')['Return'].mean(),
    'average stocks down': df[df['Direction'] == 'down'].groupby('Days After')['Return'].mean(),
    'average stocks up_median': df[df['Direction'] == 'up'].groupby('Days After')['Return'].median(),
    'average stocks down_median': df[df['Direction'] == 'down'].groupby('Days After')['Return'].median(),
    'average stocks up_std': df[df['Direction'] == 'up'].groupby('Days After')['Return'].std(),
    'average stocks down_std': df[df['Direction'] == 'down'].groupby('Days After')['Return'].std(),
    'total stocks up': df[df['Direction'] == 'up'].groupby('Days After')['Return'].count(),
    'total stocks down': df[df['Direction'] == 'down'].groupby('Days After')['Return'].count()
}

pd.DataFrame(summary).T

Days After,0,1,2,3,4,5,6,7,14,28
average stocks up,0.138745,-0.002427,0.007824,0.019239,0.025637,0.023065,0.022777,0.034134,0.064082,0.095107
average stocks down,-0.136384,0.029173,0.013228,0.030450,0.027826,0.036907,0.062424,0.054312,0.066161,0.206887
average stocks up_median,0.126903,-0.002782,0.014118,0.017542,0.027810,0.032740,0.018316,0.019399,0.065086,0.110408
average stocks down_median,-0.128396,0.020678,0.019568,0.026101,0.038933,0.039536,0.076025,0.079893,0.096115,0.201617
average stocks up_std,0.039423,0.059316,0.061105,0.095818,0.099598,0.115829,0.109873,0.115141,0.166896,0.232178
average stocks down_std,0.029105,0.067535,0.093486,0.088490,0.129886,0.147201,0.151330,0.155068,0.189754,0.341970
total stocks up,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
total stocks down,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000


In [84]:
df[(df['Direction'] == 'up') & (df['Return']>0)].groupby('Days After').count()[['Direction']] / df[df['Direction'] == 'up'].groupby('Days After').count()[['Direction']]



,Direction
Days After,
0,1.00
1,0.46
2,0.62
3,0.62
4,0.60
5,0.64
6,0.60
7,0.62
14,0.66


In [85]:
df[(df['Direction'] == 'down') & (df['Return']>0)].groupby('Days After').count()[['Direction']] / df[(df['Direction'] == 'down')].groupby('Days After').count()[['Direction']]


,Direction
Days After,
0,NaN
1,0.642857
2,0.642857
3,0.750000
4,0.571429
5,0.607143
6,0.714286
7,0.678571
14,0.678571
